<a href="https://colab.research.google.com/github/chaveroricardo/perceptron/blob/main/Notebooks/EsophagusCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score,
    roc_auc_score, roc_curve, auc, ConfusionMatrixDisplay
)
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Path to the pickle file with preprocessed images
ruta2 = '/content/drive/MyDrive/MLBusiness/'
pickle_file_path = ruta2 + "imagenes_numpy.pkl"
# Load the image data from pickle file
with open(pickle_file_path, 'rb') as f:
    X = pickle.load(f)

# Verify the data
print("Loaded image data shape:", X.shape)

Loaded image data shape: (5063,)


In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/AnIsAsPe/ClassificadorCancerEsofago/master/Datos/ClasesImagenes.csv", usecols=[1,2])
y = data['class_number'].values

print("Labels shape:", y.shape)


Labels shape: (5063,)


In [5]:
pickle_file_path = np.stack(X, axis=0)   # Takes a sequence of matrices and stacks them along
                              # a third axis to make a single array
pickle_file_path.shape


(5063, 260, 260)

In [6]:
X = pickle_file_path.reshape(5063, 67600)
X.shape

(5063, 67600)

In [7]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (3544, 67600)
Test set shape: (1519, 67600)


In [ ]:
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

# Define optimized models
models = {
    "Perceptron": Perceptron(max_iter=1000, random_state=0, verbose=False, penalty='l2', alpha=0.000005),
    "Random Forest": RandomForestClassifier(random_state=0, n_estimators=100),
    "Linear SVM": LinearSVC(random_state=0, max_iter=10000),  # Faster version of SVM for high-dimensional data
    "SGD Classifier (SVM Approximation)": SGDClassifier(loss="hinge", random_state=0, max_iter=1000, tol=1e-3)
}

# Store performance metrics for each model
model_performance = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Get probability estimates for ROC AUC
    if model_name == "Perceptron":
        y_proba = model.decision_function(X_test)
    elif model_name == "Linear SVM":
        # Since LinearSVC does not have a probability output, use decision_function
        y_proba = model.decision_function(X_test)
    else:
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None

    # Store performance
    model_performance[model_name] = {
        "Accuracy": accuracy,
        "Recall": recall,
        "Precision": precision,
        "F1 Score": f1,
        "ROC AUC": roc_auc
    }

    # Display metrics
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"F1 Score: {f1:.2f}")
    if roc_auc is not None:
        print(f"ROC AUC: {roc_auc:.2f}")
    else:
        print("ROC AUC: Not available (no probability output)")



Model: Perceptron
Accuracy: 0.90
Recall: 0.94
Precision: 0.92
F1 Score: 0.93
ROC AUC: 0.95
